In [21]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

from datasets import Dataset, NetflixData, NetflixGenreData
X_data = NetflixData(small=True)
Z_data = NetflixGenreData(small=True)

In [22]:
# down-sampling by constraints

sum_u, sum_v = X_data.X.sum

# idx_u = sum_u > 130
# idx_v = sum_v > 200

idx_u = sum_u > 40
idx_v = sum_v > 60

print("original shape:", X_data.X.shape)
print("filtered shape:", (sum(idx_u), sum(idx_v)))

X_data.sample(idx=idx_u, axis=0)
print("filtered shape:", X_data.X.shape)
X_data.sample(idx=idx_v, axis=1)
print("filtered shape:", X_data.X.shape)
Z_data.sample(idx=idx_v, axis=1)
print("filtered shape:", Z_data.X.shape)

original shape: (9980, 3362)
filtered shape: (3690, 1069)
[I] sampling with given indices
filtered shape: (3690, 3362)
[I] sampling with given indices
filtered shape: (3690, 1069)
[I] sampling with given indices
filtered shape: (22, 1069)


In [23]:
# down-sampling by random

X_data.sample(n=1000, axis=0)
print(X_data.X.shape)
idx_v = X_data.sample(n=500, axis=1)
print(X_data.X.shape)
Z_data.sample(idx=idx_v, axis=1)
print(Z_data.X.shape)

[I] sampling to size 1000
[I]   Data seed    : 1698201358
(1000, 1069)
[I] sampling to size 500
(1000, 500)
[I] sampling with given indices
(22, 500)


In [24]:
# from models import Asso
# import multiprocessing

# def asso_fit(params):
#     k, tau, w, train_set, val_set = params
#     asso = Asso(k=k, tau=tau, w=w)
#     asso.fit(train_set=train_set, display=True)
#     tpr, fpr = asso.eval(test_set=val_set, triplet=True)

#     with open('.\output\exp01.txt', "w") as f:
#         line = f'k: {k}\t tau: {tau}\t w: {w}\t tpr: {tpr}\t fpr: {fpr}\n'
#         f.write(f'{line}\n')

# params = [[k], tau_list, w_list, [ds_train], [ds_val]]
# params = list(itertools.product(*params))

# pool = multiprocessing.Pool()

# outputs_async = pool.map_async(asso_fit, params)
# outputs = outputs_async.get()
# print("Output: {}".format(outputs))

In [25]:
# k-fold

from models import Asso
import multiprocessing
import itertools
import numpy as np

n_fold = 1

for i in range(n_fold):
    fold = i + 1

    X_data.cross_validation(test_size=0, n_folds=5, current_fold=fold, seed=1997)
    pos_train_val_size = X_data.pos_train_size + X_data.pos_val_size
    X_data.cv_negative_sample(n_folds=n_fold,
                              current_fold=fold,
                              train_val_size=int(1.0 * pos_train_val_size),
                              test_size=int(0.0 * X_data.pos_test_size), 
                              seed=6666, type='popularity')

    ds_train = Dataset()
    ds_train.load_data(X=X_data.train_data, U=X_data.U, V=X_data.V)

    ds_val = Dataset()
    ds_val.load_data(X=X_data.val_data, U=X_data.U, V=X_data.V)

    ds_test = Dataset()
    ds_test.load_data(X=X_data.test_data, U=X_data.U, V=X_data.V)

    ds_train.summarize(title='training set {}'.format(fold), display=True, pixels=2, ordered=True)
    ds_val.summarize(title='validation set {}'.format(fold), display=True, pixels=2, ordered=True)
    ds_test.summarize(title='test set {}'.format(fold), display=True, pixels=2, ordered=True)


    # check tau
    tau_list = [0.20, 0.30, 0.40, 0.50, 0.60]
    for tau in tau_list:
        asso = Asso(k=5, tau=tau, w=[0.5, 0.5])
        asso._fit_prepare(train_set=ds_train, display=True)
    break


    # # grid search
    # k = 50
    # tau_list = [0.20, 0.30, 0.40, 0.50, 0.60]
    # w_list = [[0.7, 0.3], [0.8, 0.2], [0.9, 0.1]]
    # for tau in tau_list:
    #     for w in w_list:
    #         asso = Asso(k=k, tau=tau, w=w)
    #         asso._fit_prepare(train_set=ds_train, display=False)
    #         asso.fit(train_set=ds_train, display=True)
    #         tpr, fpr = asso.eval(test_set=ds_val, triplet=True)

    #         str = 'k: {}\t tau: {}\t w: {}\t tpr: {:.2f}\t fpr: {:.2f}\n'.format(k, tau, w, tpr, fpr)

    #         with open('.\output\exp01.txt', "a") as f:
    #             f.write(str)
                


[I] cross-validation, sampling positives
[I]   Data seed    : 1997
[I]   partition    : [0, 10729, 21458, 32187, 42916, 53645]
[I]   test_size    : 0
[I]   seed         : 1997
[I] getting cross-validation indices
[I]   current fold         : 1
[I]   current train size   : 42916
[I]   current val size     : 10729
[I] cross-validation, sampling negatives
[I]   Data seed    : 6666


100%|██████████| 53645/53645 [00:48<00:00, 1104.06it/s]


[I]   n_negatives  : 53645
[I]   test_size    : 0
[I]   seed         : 6666
[I] getting cross-validation indices
[I]   current fold         : 1
[I]   current train size   : 0
[I]   current val size     : 53645


ValueError: object dtype is not supported by sparse matrices